In [ ]:
# Cell 1: Install necessary packages
!pip install -q -U langgraph langchain langchain-google-genai "unstructured[all-docs]" faiss-cpu python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 19.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━

In [ ]:
# Cell 2: Configure API Keys
import os
from google.colab import userdata

# Load API key from Colab secrets
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

print("Google API Key loaded successfully.")

Google API Key loaded successfully.


In [ ]:
# Cell 3: Create the Knowledge Base & RAG Retriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# Create dummy support documents (same as before)
support_docs = {
    "technical_faq.txt": "Q: How do I reset my password? A: To reset your password, go to the login page and click 'Forgot Password'. You will receive an email with a reset link.",
    "billing_faq.txt": "Q: Where can I find my receipt? A: You can find all past receipts and invoices by logging into your account and navigating to the 'Billing History' section.",
    "general_faq.txt": "Q: What are your business hours? A: Our customer support is available 24/7 via chat. Phone support is available from 9 AM to 5 PM Eastern Time, Monday through Friday."
}

for filename, content in support_docs.items():
    with open(filename, "w") as f: f.write(content)

# Load, chunk, and create the retriever (same as before)
doc_paths = ["technical_faq.txt", "billing_faq.txt", "general_faq.txt"]
all_docs = []
for path in doc_paths:
    loader = UnstructuredFileLoader(path)
    all_docs.extend(loader.load())

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
chunks = text_splitter.split_documents(all_docs)

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever()

print("Knowledge base retriever is ready.")

Knowledge base retriever is ready.


In [ ]:
# Cell 4: Define the Agent's Tools
from langchain_core.tools import tool

# Tool 1: The RAG Tool
@tool
def knowledge_base_search(query: str) -> str:
    """
    Use this tool to answer customer questions about technical issues, billing, or company policies.
    This tool searches the company's knowledge base for relevant information.
    """
    print(f"--- Searching knowledge base for: {query} ---")
    docs = retriever.invoke(query)
    return "\n".join([doc.page_content for doc in docs])

# Tool 2: The Escalation Tool
@tool
def escalate_to_human(query: str) -> str:
    """
    Use this tool ONLY when the user is angry, frustrated, or expressing strong negative emotions.
    This tool escalates the query to a human support agent.
    """
    print(f"--- Escalating to human: {query} ---")
    return "The user's query has been escalated to a human agent. Please stand by."

# Tool 3: A General Fallback Tool
@tool
def answer_directly(query: str) -> str:
    """
    Use this tool as a last resort if no other tool is suitable for answering a general greeting or a question you can answer from your own knowledge.
    """
    print(f"--- Answering directly for: {query} ---")
    return "This is a direct answer from the support agent."

# Compile the list of tools for the agent
tools = [knowledge_base_search, escalate_to_human, answer_directly]

In [ ]:
# Cell 5: Create the Single, Efficient Router Agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor

# Use a free Gemini model. gemini-1.5-flash is fast and efficient.
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0, convert_system_message_to_human=True)

# The powerful system prompt that guides the agent's decision-making
system_prompt = """
You are an intelligent customer support routing agent. Your primary goal is to analyze a user's query and choose the single best tool to handle it.

Follow these steps in your reasoning process:
1.  **Analyze Sentiment:** First, determine the user's sentiment. Is the user angry, frustrated, or expressing negative emotions?
2.  **Categorize Query:** Second, determine the query's category. Is it a technical question, a billing question, or a general inquiry?
3.  **Select ONE Tool:** Based on your analysis, you must select one and only one of the following tools:

    *   `escalate_to_human`: Choose this tool ONLY if the sentiment is clearly Negative.
    *   `knowledge_base_search`: Choose this tool if the sentiment is Neutral or Positive AND the query is a specific question about technical, billing, or policy matters.
    *   `answer_directly`: Choose this tool for simple greetings like "hello" or questions that don't fit other categories.

Your final output should be the result of the tool you have chosen.
"""

# Create the agent
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("Efficient Router Agent created successfully.")

Efficient Router Agent created successfully.


In [ ]:
# Cell 6: Run and Test the Efficient System

# --- Test Cases ---

# Test Case 1: Should trigger the ESCALATE tool
print("--- TEST CASE 1: NEGATIVE SENTIMENT ---")
query1 = "My internet is so slow, this is unacceptable! I want to cancel immediately."
response1 = agent_executor.invoke({"input": query1})
print(f"\nQuery: {query1}\nFinal Response: {response1['output']}\n" + "="*50)


# Test Case 2: Should trigger the RAG (knowledge_base_search) tool
print("\n--- TEST CASE 2: BILLING QUESTION ---")
query2 = "How can I get a copy of my last invoice?"
response2 = agent_executor.invoke({"input": query2})
print(f"\nQuery: {query2}\nFinal Response: {response2['output']}\n" + "="*50)


# Test Case 3: Should also trigger the RAG tool
print("\n--- TEST CASE 3: TECHNICAL QUESTION ---")
query3 = "I forgot my password, what do I do?"
response3 = agent_executor.invoke({"input": query3})
print(f"\nQuery: {query3}\nFinal Response: {response3['output']}\n" + "="*50)

# Test Case 4: Should trigger the DIRECT ANSWER tool
print("\n--- TEST CASE 4: SIMPLE GREETING ---")
query4 = "hi there"
response4 = agent_executor.invoke({"input": query4})
print(f"\nQuery: {query4}\nFinal Response: {response4['output']}\n" + "="*50)

--- TEST CASE 1: NEGATIVE SENTIMENT ---


> Entering new AgentExecutor chain...


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Invoking: `escalate_to_human` with `{'query': 'My internet is so slow, this is unacceptable! I want to cancel immediately.'}`


--- Escalating to human: My internet is so slow, this is unacceptable! I want to cancel immediately. ---
The user's query has been escalated to a human agent. Please stand by.

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


The customer is clearly angry and wants to cancel their service.  Therefore, the `escalate_to_human` tool is the most appropriate choice.


> Finished chain.

Query: My internet is so slow, this is unacceptable! I want to cancel immediately.
Final Response: The customer is clearly angry and wants to cancel their service.  Therefore, the `escalate_to_human` tool is the most appropriate choice.


--- TEST CASE 2: BILLING QUESTION ---


> Entering new AgentExecutor chain...


...

To reset your password, go to the login page and click 'Forgot Password'. You will receive an email with a reset link.


> Finished chain.

Query: I forgot my password, what do I do?
Final Response: To reset your password, go to the login page and click 'Forgot Password'. You will receive an email with a reset link.


--- TEST CASE 4: SIMPLE GREETING ---


> Entering new AgentExecutor chain...


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Invoking: `answer_directly` with `{'query': 'hi there'}`


--- Answering directly for: hi there ---
This is a direct answer from the support agent.

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


OK.  The user said "hi there". This is a simple greeting.  I will use the `answer_directly` tool.


> Finished chain.

Query: hi there
Final Response: OK.  The user said "hi there". This is a simple greeting.  I will use the `answer_directly` tool.



In [ ]:
# Upgrade the langchain package to the latest version
!pip install --upgrade langchain

# Re-run the cell to see if the error is resolved

In [ ]:
# Install langchain-community
!pip install langchain-community

# Re-run the cell above (Cell 3) after this installation is complete

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00


In [ ]:
# Cell 1: Install necessary packages
!pip install -q -U langgraph langchain langchain-google-genai faiss-cpu python-dotenv beautifulsoup4

In [ ]:
# Cell 2: Configure API Keys
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

print("Google API Key loaded successfully.")

Google API Key loaded successfully.


In [ ]:
# Cell 3: Create Knowledge Base from Real-World Stripe FAQs
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings

print("--- Fetching real-world data from Stripe's documentation... ---")

# Define the URLs for our knowledge base
# These pages cover billing, technical integration, and general platform questions.
stripe_faq_urls = [
    "https://stripe.com/docs/testing", # Technical: How to test payments
    "https://stripe.com/docs/billing/subscriptions/overview", # Billing: How subscriptions work
    "https://stripe.com/docs/payments/handling-payment-events", # Technical: Handling webhooks
    "https://stripe.com/docs/disputes", # Billing/General: Handling disputes
    "https://stripe.com/docs/security/guide" # General: Security at Stripe
]

# Use WebBaseLoader to scrape the content from these pages
loader = WebBaseLoader(stripe_faq_urls)
loader.requests_per_second = 1 # Be respectful of their servers
docs = loader.load()

print(f"--- Successfully loaded {len(docs)} documents from the web. ---")

# Split the loaded documents into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150
)
chunks = text_splitter.split_documents(docs)

print(f"--- Created {len(chunks)} document chunks for the knowledge base. ---")

# Create embeddings and the FAISS vector store
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever()

print("--- Real-world knowledge base and retriever are ready. ---")

--- Fetching real-world data from Stripe's documentation... ---
--- Successfully loaded 5 documents from the web. ---
--- Created 113 document chunks for the knowledge base. ---
--- Real-world knowledge base and retriever are ready. ---


In [ ]:
# Cell 4: Define the Agent's Tools
from langchain_core.tools import tool

# Tool 1: The RAG Tool (now searches Stripe data)
@tool
def knowledge_base_search(query: str) -> str:
    """
    Use this tool to answer customer questions about technical integrations (testing, webhooks),
    billing (subscriptions, disputes), or security policies at Stripe.
    """
    print(f"--- Searching Stripe knowledge base for: {query} ---")
    docs = retriever.invoke(query)
    return "\n".join([doc.page_content for doc in docs])

# Tool 2: The Escalation Tool
@tool
def escalate_to_human(query: str) -> str:
    """
    Use this tool ONLY when the user is angry, frustrated, or expressing strong negative emotions.
    """
    print(f"--- Escalating to human: {query} ---")
    return "The user's query has been escalated to a human agent. Please stand by."

# Tool 3: A General Fallback Tool
@tool
def answer_directly(query: str) -> str:
    """
    Use this tool as a last resort for simple greetings or questions you can answer from your own knowledge.
    """
    print(f"--- Answering directly for: {query} ---")
    return "This is a direct answer from the support agent."

tools = [knowledge_base_search, escalate_to_human, answer_directly]

In [ ]:
# Cell 5: Create the Single, Efficient Router Agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0, convert_system_message_to_human=True)

system_prompt = """
You are an intelligent customer support routing agent for Stripe. Your primary goal is to analyze a user's query and choose the single best tool to handle it.

Follow these steps in your reasoning process:
1.  **Analyze Sentiment:** First, determine the user's sentiment. Is the user angry or frustrated?
2.  **Select ONE Tool:** Based on the user's needs, you must select one and only one of the following tools:

    *   `escalate_to_human`: Choose this tool ONLY if the sentiment is clearly Negative.
    *   `knowledge_base_search`: Choose this tool for ANY question about how Stripe works, including testing, billing, subscriptions, security, disputes, or webhooks. This should be your default choice for most questions.
    *   `answer_directly`: Choose this tool ONLY for simple greetings like "hello".

Your final output should be the result of the tool you have chosen.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("Efficient Router Agent for Stripe Support created successfully.")

Efficient Router Agent for Stripe Support created successfully.


In [ ]:
# Cell 6: Run and Test the System with Real-World Queries

# --- Test Cases ---

# Test Case 1: Should trigger ESCALATE
print("--- TEST CASE 1: NEGATIVE SENTIMENT ---")
query1 = "I can't believe you charged me twice for my subscription! This is outrageous, fix it now!"
response1 = agent_executor.invoke({"input": query1})
print(f"\nQuery: {query1}\nFinal Response: {response1['output']}\n" + "="*50)

# Test Case 2: Should trigger RAG (knowledge_base_search) for a technical question
print("\n--- TEST CASE 2: TECHNICAL QUESTION ---")
query2 = "How do I test different payment failure scenarios for my integration?"
response2 = agent_executor.invoke({"input": query2})
print(f"\nQuery: {query2}\nFinal Response: {response2['output']}\n" + "="*50)

# Test Case 3: Should trigger RAG (knowledge_base_search) for a billing question
print("\n--- TEST CASE 3: BILLING QUESTION ---")
query3 = "What happens if a customer disputes a charge on their credit card?"
response3 = agent_executor.invoke({"input": query3})
print(f"\nQuery: {query3}\nFinal Response: {response3['output']}\n" + "="*50)

--- TEST CASE 1: NEGATIVE SENTIMENT ---


> Entering new AgentExecutor chain...


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Invoking: `escalate_to_human` with `{'query': "I can't believe you charged me twice for my subscription! This is outrageous, fix it now!"}`


--- Escalating to human: I can't believe you charged me twice for my subscription! This is outrageous, fix it now! ---
The user's query has been escalated to a human agent. Please stand by.

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


The user's query has been escalated to a human agent. Please stand by.


> Finished chain.

Query: I can't believe you charged me twice for my subscription! This is outrageous, fix it now!
Final Response: The user's query has been escalated to a human agent. Please stand by.


--- TEST CASE 2: TECHNICAL QUESTION ---


> Entering new AgentExecutor chain...


...

The provided text describes how to test payment integrations using Stripe's test environment and test cards.  It covers simulating successful and failed payments, handling card errors, and testing webhooks.


> Finished chain.

Query: How do I test different payment failure scenarios for my integration?
Final Response: The provided text describes how to test payment integrations using Stripe's test environment and test cards.  It covers simulating successful and failed payments, handling card errors, and testing webhooks.


--- TEST CASE 3: BILLING QUESTION ---


> Entering new AgentExecutor chain...


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Invoking: `knowledge_base_search` with `{'query': 'What happens if a customer disputes a charge on their credit card?'}`


--- Searching Stripe knowledge base for: What happens if a customer disputes a charge on their credit card? ---
them.A dispute (also known as a chargeback) occurs when a cardholder questions your payment with their card issuer.To process a chargeback, the issuer creates a formal dispute on the card network, which immediately reverses the payment. This pulls the money for the payment—as well as one or more network dispute fees—from Stripe. After that, Stripe debits your balance for the payment amount and dispute fee.To help our users submit the best possible response for each dispute, Stripe guides you through the process within the Dashboard. Here, you can provide the appropriate text and images for the dispute reason, and your counterargument. If you need help with a dispute, contact support.Handle disputes Respond to disputesLearn how to challenge or accept a di

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


When a customer disputes a charge, it's called a chargeback.  The card issuer reverses the payment, and Stripe debits your account for the payment and any dispute fees.  Stripe guides you through the process in the Dashboard, where you can provide evidence to support your case.  If you need help, contact support.


> Finished chain.

Query: What happens if a customer disputes a charge on their credit card?
Final Response: When a customer disputes a charge, it's called a chargeback.  The card issuer reverses the payment, and Stripe debits your account for the payment and any dispute fees.  Stripe guides you through the process in the Dashboard, where you can provide evidence to support your case.  If you need help, contact support.

